In [88]:
H = matrix([
    [1, 1, 1, 0, 0, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 1, 1, 1, 0, 1, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 1, 1, 0]
])


y = list(map(float, ['-0.35', '-0.43', '-0.73', '-0.05', '0.2', '0.98', '-0.73', '0.36', '-0.45', '0.58']))

In [89]:
from sage.coding.ldpc import LDPCCode, LDPCDecoder

ModuleNotFoundError: No module named 'sage.coding.ldpc'

In [51]:
N_m = [[i for i, e in enumerate(row) if e == 1] for row in H]
M_n = [[i for i, e in enumerate(column) if e == 1] for column in H.columns()]

In [52]:
N_m

[[0, 1, 2, 5, 6, 9],
 [0, 2, 4, 5, 7, 8],
 [2, 3, 4, 6, 8, 9],
 [1, 3, 4, 5, 7, 9],
 [0, 1, 3, 6, 7, 8]]

In [53]:
M_n

[[0, 1, 4],
 [0, 3, 4],
 [0, 1, 2],
 [2, 3, 4],
 [1, 2, 3],
 [0, 1, 3],
 [0, 2, 4],
 [1, 3, 4],
 [1, 2, 4],
 [0, 2, 3]]

In [54]:
n, m = H.dimensions()

In [55]:
L = [[0 for _ in range(m)] for _ in range(n)]

In [56]:
import copy 
def findL(L):
    LL = copy.deepcopy(L) 
    for i in range(n): 
        # iterate each row 
        c = 0 
        for j in range(m): 
            # each value in row       
            tmp_place = int(H[i][j])
            if tmp_place != 0: 
                LL[i][j] = y[N_m[i][c]] # replace with y val. 
                c += 1
            else: 
                LL[i][j] = 0 
    return LL 

L = findL(L)

In [58]:
L

[[-0.35, -0.43, -0.73, 0, 0, 0.98, -0.73, 0, 0, 0.58],
 [-0.35, 0, -0.73, 0, 0.2, 0.98, 0, 0.36, -0.45, 0],
 [0, 0, -0.73, -0.05, 0.2, 0, -0.73, 0, -0.45, 0.58],
 [0, -0.43, 0, -0.05, 0.2, 0.98, 0, 0.36, 0, 0.58],
 [-0.35, -0.43, 0, -0.05, 0, 0, -0.73, 0.36, -0.45, 0]]

In [140]:
def get_connected_bits(H, i):
    """
    Computes list of elements of N(i)-j:
    List of variables (bits) connected to Parity node i.

    """
    m, n = H.shape
    return [a for a in range(n) if H[i, a]]


def get_connected_nodes(tH, j):
    """
    Computes list of elements of M(j):
    List of nodes (PC equations) connecting variable j.

    """
    return get_connected_bits(tH, j)

def get_Nm_M_n(H):
    m, n = H.shape
    tH = np.transpose(H)
    Bits = [Bits2i(H, i) for i in range(m)]
    Nodes = [Nodes2j(tH, j) for j in range(n)]
    return Bits, Nodes

In [141]:
import numpy as np

def ldpc_decode_belief_propagation(symbol, H, max_iter=2):
    """
    A LDPC decoder based on the belief propagation method.

    Parameters:
        symbol: received symbols
        H: check matrix
        max_iter: maximum number of iterations (default: 2)

    Returns:
        decoded message bits
    """
    # Extract matrix dimensions
    num_check_bits, num_coding_bits = H.shape
    num_message_bits = num_coding_bits - num_check_bits

    # Initialize variables
    beta = np.zeros((num_check_bits, num_coding_bits))
    alpha = np.zeros((num_check_bits, num_coding_bits))
    decide = np.zeros(num_coding_bits)

    # Find the nonzero elements
    N_m, H_m = get_Nm_M_n(H)
    # Initialize beta matrix
    for check_id in range(num_check_bits):
        connected_bits = N_m[check_id]
        beta[check_id, connected_bits] = symbol[connected_bits]

    # Message update
    for iteration in range(max_iter):
        # Step 2: Horizontal message passing
        for bit_id in range(num_coding_bits):
            connected_checks = H_m[bit_id]
            for check_id in connected_checks:
                connected_bits = N_m[check_id]
                # Calculate the product of the tanh functions for the connected bits
                X = np.prod(np.tanh(0.5 * beta[check_id, connected_bits]))
                # Compute the alpha values (check node to bit node messages)
                alpha[check_id, bit_id] = 2 * np.arctanh(X)

            # Step 2: Vertical message passing
            Mij = [check_id for check_id in connected_checks if check_id != bit_id]
            # Compute the beta values (bit node to check node messages)
            beta[:, bit_id] = symbol[bit_id] + np.sum(alpha[Mij, bit_id])

        # Step 3: Compute the updated messages and decisions
        for bit_id in range(num_coding_bits):
            connected_checks = H_m[bit_id]
            # Compute the updated message for the bit node
            m_bit = np.sum(alpha[connected_checks, bit_id]) + np.sum(beta[connected_checks, bit_id])
            # Compute the decision for the bit node
            decide[bit_id] = symbol[bit_id] + m_bit

    # Soft decision: Decode the message bits based on the decisions
    decoded_message_bits = np.zeros(num_message_bits)
    for i in range(num_message_bits):
        if decide[i + num_check_bits] < 0:
            decoded_message_bits[i] = 1

    return decoded_message_bits


In [142]:
H = np.array([
    [1, 1, 1, 0, 0, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 1, 1, 1, 0, 1, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 1, 1, 0]
])
y = np.array(y)

In [143]:
m = list(map(int, ldpc_decode_belief_propagation(y, H, max_iter=1)))

In [144]:
m

[0, 1, 0, 1, 0]